In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Loading data

In [3]:
NUM_SAMPLES = 10000

input_texts = []
target_texts = []
target_texts_input = []

In [10]:
t = 0

with open("../neural_machine_translation/data/hin.txt") as f:
    for line in f:
        
        # keeping only limited number of samples
        t += 1
        if t > NUM_SAMPLES:
            break
            
        if '\t' not in line:
            continue
            
        input_text, translation, *rest = line.rstrip().split('\t')
        
        target_text = translation + ' <eos>'
        target_text_input = '<sos> ' + translation
        
        input_texts.append(input_text)
        target_texts.append(target_text)
        target_texts_input.append(target_text_input)
    

In [11]:
target_texts_input[:15]

['<sos> वाह!',
 '<sos> झुको!',
 '<sos> बतख़!',
 '<sos> बचाओ!',
 '<sos> उछलो.',
 '<sos> कूदो.',
 '<sos> छलांग.',
 '<sos> नमस्ते।',
 '<sos> नमस्कार।',
 '<sos> वाह-वाह!',
 '<sos> चियर्स!',
 '<sos> सांस छोड़।',
 '<sos> सांस छोड़ो।',
 '<sos> समझे कि नहीं?',
 '<sos> मैं ठीक हूँ।']

### Tokenizers and word to index mappings

#### I have two languages to deal with, hence need two different tokenizers

In [18]:
print(input_texts[:5])

['Wow!', 'Duck!', 'Duck!', 'Help!', 'Jump.']


In [19]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 20000
MAX_SEQ_LEN = 100

In [47]:
# tokenizer for English language
tokenizer_inputs = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)
print(input_sequences[:7])


# word_to_index mapping 
from itertools import islice

word_to_idx_inputs = tokenizer_inputs.word_index

first_10_mapping_inputs = dict(islice(word_to_idx_inputs.items(), 10))
first_10_mapping_inputs


# maximum length of input sequences
max_len_inputs = max(len(i) for i in input_sequences)
print("Maximum length of input sequences:",max_len_inputs)

[[1326], [949], [949], [83], [582], [582], [582]]
Maximum length of input sequences: 22
